In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

apandeyucsd_cse251b_path = kagglehub.dataset_download('apandeyucsd/cse251b')

print('Data source import complete.')


100%|██████████| 0.98G/0.98G [00:11<00:00, 93.1MB/s]

Extracting files...


Data source import complete.


In [ ]:
print(apandeyucsd_cse251b_path)

/root/.cache/kagglehub/datasets/apandeyucsd/cse251b/versions/1


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

# Load raw .npz data
train_file = np.load(apandeyucsd_cse251b_path+ '/train.npz')
train_data = train_file['data']       # shape (N, 50, 110, 6)
test_file  = np.load(apandeyucsd_cse251b_path + '/test_input.npz')
test_data  = test_file['data']

# Feature-label preparation: flatten agent 0’s history/future
def prepare_xy(data):
    # data: (N, agents=50, hist+fut=110, dims=6)
    # pick agent 0, positions only
    hist = data[:, 0, :50, :2]    # (N,50,2)
    fut  = data[:, 0, 50:, :2]    # (N,60,2)
    X = hist.reshape(len(data), -1)   # (N,100)
    y = fut .reshape(len(data), -1)   # (N,120)
    return X, y

# train/val split
train_size = int(0.8 * len(train_data))
X_train, y_train = prepare_xy(train_data[:train_size])
X_val,   y_val   = prepare_xy(train_data[train_size:])

# Build simple TensorDatasets & DataLoaders
batch_size = 32
train_ds = TensorDataset(torch.from_numpy(X_train).float(),
                         torch.from_numpy(y_train).float())
val_ds   = TensorDataset(torch.from_numpy(X_val  ).float(),
                         torch.from_numpy(y_val  ).float())

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size)

# Define a linear‐regression model
class LinearForecast(nn.Module):
    def __init__(self, in_dim=100, out_dim=120):
        super().__init__()
        self.linear = nn.Linear(in_dim, out_dim)
    def forward(self, x):
        return self.linear(x)

model = LinearForecast()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Training loop
num_epochs = 20
for epoch in range(1, num_epochs+1):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        pred = model(xb)
        loss = criterion(pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * xb.size(0)
    avg_train_loss = total_loss / len(train_loader.dataset)

    model.eval()
    total_val = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            total_val += criterion(model(xb), yb).item() * xb.size(0)
    avg_val_loss = total_val / len(val_loader.dataset)

    print(f"Epoch {epoch:02d}: train MSE={avg_train_loss:.4f}, val MSE={avg_val_loss:.4f}")

# Inference on test set
X_test, _ = prepare_xy(test_data)
test_loader = DataLoader(
    TensorDataset(torch.from_numpy(X_test).float()),
    batch_size=batch_size
)

model.eval()
predictions = []
with torch.no_grad():
    for (xb,) in test_loader:
        out = model(xb)                        # (B,120)
        out = out.reshape(-1, 60, 2) * 10.0    # un-normalize
        predictions.append(out.cpu().numpy())
predictions = np.vstack(predictions)         # (N_test, 60, 2)



Epoch 01: train MSE=528629.8208, val MSE=844.8843
Epoch 02: train MSE=862.0956, val MSE=849.5536
Epoch 03: train MSE=879.8954, val MSE=886.2119
Epoch 04: train MSE=889.9458, val MSE=904.3873
Epoch 05: train MSE=904.9181, val MSE=864.0377
Epoch 06: train MSE=905.5759, val MSE=863.5824
Epoch 07: train MSE=915.9065, val MSE=854.4289
Epoch 08: train MSE=916.8784, val MSE=905.9896
Epoch 09: train MSE=955.0869, val MSE=885.6002
Epoch 10: train MSE=929.3858, val MSE=824.0100
Epoch 11: train MSE=941.8441, val MSE=906.3152
Epoch 12: train MSE=955.7220, val MSE=934.6329
Epoch 13: train MSE=1029.6585, val MSE=866.4145
Epoch 14: train MSE=966.3759, val MSE=880.1479
Epoch 15: train MSE=952.9590, val MSE=808.8428
Epoch 16: train MSE=993.8412, val MSE=842.6538
Epoch 17: train MSE=1065.4067, val MSE=979.6435
Epoch 18: train MSE=973.6157, val MSE=946.7229
Epoch 19: train MSE=1041.2218, val MSE=929.9599
Epoch 20: train MSE=1221.5481, val MSE=769.8255


In [ ]:
# conversion into csv
# single stream of rows (N_test*60 rows, 2 columns)
df = pd.DataFrame(
    predictions.reshape(-1, 2),      # → (N_test*60, 2)
    columns=['x','y']
)

# name index column
df.index.name = 'index'

# save to CSV
submission_filename = 'lin_reg_submission.csv'
df.to_csv(submission_filename)

# download in colab
from google.colab import files
files.download(submission_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>